# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bluff,-46.6000,168.3333,13.22,78,23,4.35,NZ,1680338839
1,1,gat,31.6100,34.7642,18.38,14,2,3.30,IL,1680338840
2,2,waipawa,-41.4122,175.5153,15.09,89,100,3.20,NZ,1680338840
3,3,iqaluit,63.7506,-68.5145,-12.15,85,100,2.57,CA,1680338841
4,4,tasiilaq,65.6145,-37.6368,-3.91,76,100,2.43,GL,1680338841
5,5,bredasdorp,-34.5322,20.0403,22.01,57,4,5.32,ZA,1680338841
6,6,hobart,-42.8794,147.3294,13.92,66,75,5.14,AU,1680338826
7,7,ouallam,14.3188,2.0866,32.05,7,12,6.74,NE,1680338842
8,8,elat,29.5581,34.9482,18.91,32,0,7.20,IL,1680338842
9,9,castro,-24.7911,-50.0119,15.23,93,0,1.33,BR,1680338842


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["City", "Lat","Lng","Max Temp","Humidity","Cloudiness","Wind Speed","Country"]
)
# Display the map plot_1
map_plot_1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Max Temp,Cloudiness,Wind Speed,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Humidity"] < 30) & ((city_data_df["Max Temp"] < 28) & (city_data_df["Max Temp"] > 18))]

# Drop any rows with null values
city_data_df.dropna(inplace=True)


# Display sample data
city_data_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,gat,31.6100,34.7642,18.38,14,2,3.30,IL,1680338840
54,54,komsomolskiy,40.4272,71.7189,24.40,22,94,3.09,UZ,1680338858
72,72,jalu,29.0331,21.5482,22.38,19,0,4.43,LY,1680338865
259,259,ajdabiya,30.7554,20.2263,22.61,18,4,5.35,LY,1680338931
264,264,gazojak,41.1875,61.4036,24.08,12,4,5.66,TM,1680338769
276,276,tahta,26.7693,31.5021,22.06,21,0,6.53,EG,1680338936
330,330,saint-louis,16.3333,-15.0000,25.80,7,0,6.34,SN,1680338955
339,339,nenjiang,49.1833,125.2167,20.62,17,98,7.88,CN,1680338958
371,371,seoul,37.5683,126.9778,25.66,19,20,5.14,KR,1680338731
398,398,buraydah,26.3260,43.9750,19.25,14,0,3.45,SA,1680338979


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City","Country", "Lat","Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(5,"Hotel Name",'')
# city_data_df.append["Hotel Name"]

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
1,gat,IL,31.6100,34.7642,14,
54,komsomolskiy,UZ,40.4272,71.7189,22,
72,jalu,LY,29.0331,21.5482,19,
259,ajdabiya,LY,30.7554,20.2263,18,
264,gazojak,TM,41.1875,61.4036,12,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
# Set parameters to search for a hotel
radius = 100000
params = {    
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    req=requests.Request("GET",base_url,params=params)
    prepared_request=req.prepare()
    print(prepared_request.url)
    
    # continue
    # Make an API request using the params dictionary
    name_address = requests.Session().send(prepared_request)
    print (str(name_address.status_code))
    # name_address = requests.get(base_url, params=params)

#   # https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle:
#    -0.07071648508463113,51.50848194136378,1000&bias=proximity:-0.07071648508463113,51.50848194136378&limit=20&apiKey=12a2695b4f7146a6bf8cba58cdb36982
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    print(name_address)
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data

hotel_df

Starting hotel search
https://api.geoapify.com/v2/places?categories=accommodation.hotel&apiKey=12a2695b4f7146a6bf8cba58cdb36982&filter=circle%3A34.7642%2C31.61%2C100000&bias=proximity%3A34.7642%2C31.61
200
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'מלון גולדן טאוור', 'country': 'Israel', 'country_code': 'il', 'state': 'South District', 'county': 'Ashkelon Subdistrict', 'city': 'Ashkelon', 'postcode': '7875809', 'district': 'Eshkoly Paz', 'street': 'Yonatan Fridan', 'lon': 34.56177935308921, 'lat': 31.6748687, 'formatted': 'Golden Tower Hotel, Yonatan Fridan, 7875809 Ashkelon, Israel', 'address_line1': 'Golden Tower Hotel', 'address_line2': 'Yonatan Fridan, 7875809 Ashkelon, Israel', 'categories': ['accommodation', 'accommodation.hotel'], 'details': ['details'], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database Licence', 'url': 'https://www.openstreetmap.org/copyright', 'raw'

,City,Country,Lat,Lng,Humidity,Hotel Name
1,gat,IL,31.6100,34.7642,14,מלון גולדן טאוור
54,komsomolskiy,UZ,40.4272,71.7189,22,"Hotel ""Ikat House"""
72,jalu,LY,29.0331,21.5482,19,فندق جالو السياحي
259,ajdabiya,LY,30.7554,20.2263,18,فندق المدينة
264,gazojak,TM,41.1875,61.4036,12,Жайхун
276,tahta,EG,26.7693,31.5021,21,Al Safa Hotel
330,saint-louis,SN,16.3333,-15.0000,7,La Cour du Fleuve
339,nenjiang,CN,49.1833,125.2167,17,No hotel found
371,seoul,KR,37.5683,126.9778,19,뉴국제호텔
398,buraydah,SA,26.3260,43.9750,14,دنا للوحدات السكنية


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE